In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import os
import pickle
import tarfile
import urllib.request

# Step 3: Download and extract the CIFAR-10 dataset
dataset_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
dataset_path = "/content/cifar-10-python.tar.gz"
extracted_path = "/content/cifar-10-batches-py"

# Download the dataset
urllib.request.urlretrieve(dataset_url, dataset_path)

# Extract the dataset
with tarfile.open(dataset_path, "r:gz") as tar:
    tar.extractall(path="/content")

# Step 4: Load and preprocess the CIFAR-10 dataset manually
def load_batch(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    data = dict[b'data']
    labels = dict[b'labels']
    data = data.reshape(len(data), 3, 32, 32).transpose(0, 2, 3, 1)
    return data, labels

# Load training batches
x_train = []
y_train = []

for i in range(1, 6):
    file_path = os.path.join(extracted_path, f'data_batch_{i}')
    data, labels = load_batch(file_path)
    x_train.append(data)
    y_train.append(labels)

x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)

# Load test batch
file_path = os.path.join(extracted_path, 'test_batch')
x_test, y_test = load_batch(file_path)

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Step 5: Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Step 6: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 7: Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test))

# Step 8: Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# Save the model
model.save('/content/cifar10_trained_model.h5')

# Step 9: Load your trained model and make predictions on a custom image
model = tf.keras.models.load_model('/content/cifar10_trained_model.h5')

# Upload your custom image to Colab
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is named 'dog2.jpg'
image_path = '/content/truck.jpg'

# Load your own image
import cv2
image = cv2.imread(image_path)

# Preprocess your image
image = cv2.resize(image, (32, 32))
image = image.astype('float32') / 255.0
image = np.expand_dims(image, axis=0)

# Make predictions
predictions = model.predict(image)
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
predicted_class = class_labels[np.argmax(predictions)]

print("Predicted Class:", predicted_class)
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
predicted_class = class_labels[np.argmax(predictions)]

print("Predicted Class:", predicted_class)

Epoch 1/20
782/782 [==============================] - 7s 7ms/step - loss: 1.8051 - accuracy: 0.3256 - val_loss: 1.4623 - val_accuracy: 0.4564
Epoch 2/20
782/782 [==============================] - 5s 6ms/step - loss: 1.4873 - accuracy: 0.4597 - val_loss: 1.3563 - val_accuracy: 0.5173
Epoch 3/20
782/782 [==============================] - 4s 5ms/step - loss: 1.3565 - accuracy: 0.5106 - val_loss: 1.2403 - val_accuracy: 0.5525
Epoch 4/20
782/782 [==============================] - 5s 6ms/step - loss: 1.2785 - accuracy: 0.5450 - val_loss: 1.1093 - val_accuracy: 0.6113
Epoch 5/20
782/782 [==============================] - 4s 5ms/step - loss: 1.2041 - accuracy: 0.5727 - val_loss: 1.0448 - val_accuracy: 0.6351
Epoch 6/20
782/782 [==============================] - 4s 5ms/step - loss: 1.1594 - accuracy: 0.5899 - val_loss: 0.9730 - val_accuracy: 0.6586
Epoch 7/20
782/782 [==============================] - 5s 7ms/step - loss: 1.1284 - accuracy: 0.6039 - val_loss: 0.9683 - val_accuracy: 0.6745
Epoch 

Saving truck.jpg to truck.jpg
1/1 [==============================] - 0s 313ms/step
Predicted Class: truck
Predicted Class: truck
